In [6]:
# Use built-in sqlite3; no pip install needed
import sqlite3, sys

print(sys.version)                  # Python version
print(sqlite3.sqlite_version)       # Linked SQLite version

con = sqlite3.connect(":memory:")
con

3.13.0 (main, Oct 16 2024, 00:33:24) [MSC v.1929 64 bit (AMD64)]
3.46.0


In [8]:
# Robustly locate mock_energy_metadata.json and load into SQLite (no pysqlite3 needed)
import sqlite3, json
from pathlib import Path

print("CWD:", Path.cwd())

def find_json():
    # Try common locations relative to notebook start dir
    candidates = [
        Path("catalog_agent/data/mock_energy_metadata.json"),
        Path("data/mock_energy_metadata.json"),
        Path("../data/mock_energy_metadata.json"),
        Path("../../catalog_agent/data/mock_energy_metadata.json"),
    ]
    for p in candidates:
        if p.exists():
            return p.resolve()
    raise FileNotFoundError("mock_energy_metadata.json not found. Checked:\n" + "\n".join(str(p) for p in candidates))

json_path = find_json()
db_path = json_path.parent / "metadata.sqlite"
print("JSON:", json_path)
print("DB  :", db_path)

records = json.loads(json_path.read_text(encoding="utf-8"))

con = sqlite3.connect(db_path)
con.execute("PRAGMA foreign_keys = ON")

con.execute("""
CREATE TABLE IF NOT EXISTS energy_metadata (
    dataset_id TEXT PRIMARY KEY,
    title TEXT,
    description TEXT,
    publisher TEXT,
    source_system TEXT,
    data_category TEXT,
    granularity TEXT,
    geographical_scope TEXT,
    temporal_coverage_start TEXT,
    temporal_coverage_end TEXT,
    format TEXT,
    size_mb REAL,
    update_frequency TEXT,
    license TEXT,
    created_at TEXT,
    last_modified TEXT,
    contact TEXT,
    keywords_json TEXT,
    quality_score REAL
)
""")

rows = []
for r in records:
    rows.append((
        r.get("dataset_id"),
        r.get("title"),
        r.get("description"),
        r.get("publisher"),
        r.get("source_system"),
        r.get("data_category"),
        r.get("granularity"),
        r.get("geographical_scope"),
        r.get("temporal_coverage_start"),
        r.get("temporal_coverage_end"),
        r.get("format"),
        r.get("size_mb"),
        r.get("update_frequency"),
        r.get("license"),
        r.get("created_at"),
        r.get("last_modified"),
        r.get("contact"),
        json.dumps(r.get("keywords", []), ensure_ascii=False),
        r.get("quality_score"),
    ))

con.executemany("""
INSERT INTO energy_metadata (
  dataset_id,title,description,publisher,source_system,data_category,granularity,geographical_scope,
  temporal_coverage_start,temporal_coverage_end,format,size_mb,update_frequency,license,
  created_at,last_modified,contact,keywords_json,quality_score
) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
ON CONFLICT(dataset_id) DO UPDATE SET
  title=excluded.title,
  description=excluded.description,
  publisher=excluded.publisher,
  source_system=excluded.source_system,
  data_category=excluded.data_category,
  granularity=excluded.granularity,
  geographical_scope=excluded.geographical_scope,
  temporal_coverage_start=excluded.temporal_coverage_start,
  temporal_coverage_end=excluded.temporal_coverage_end,
  format=excluded.format,
  size_mb=excluded.size_mb,
  update_frequency=excluded.update_frequency,
  license=excluded.license,
  created_at=excluded.created_at,
  last_modified=excluded.last_modified,
  contact=excluded.contact,
  keywords_json=excluded.keywords_json,
  quality_score=excluded.quality_score
""", rows)

con.execute("""
CREATE TABLE IF NOT EXISTS energy_keyword (
    dataset_id TEXT NOT NULL,
    keyword TEXT NOT NULL,
    PRIMARY KEY (dataset_id, keyword),
    FOREIGN KEY (dataset_id) REFERENCES energy_metadata(dataset_id) ON DELETE CASCADE
)
""")

kw_rows = []
for r in records:
    did = r.get("dataset_id")
    for kw in r.get("keywords", []) or []:
        kw_rows.append((did, kw))
con.executemany("INSERT OR IGNORE INTO energy_keyword (dataset_id, keyword) VALUES (?,?)", kw_rows)

con.commit()
con.close()
print("Import fertig:", db_path)

CWD: c:\Projekte\fedcatalog_agent\catalog_agent\tests
JSON: C:\Projekte\fedcatalog_agent\catalog_agent\data\mock_energy_metadata.json
DB  : C:\Projekte\fedcatalog_agent\catalog_agent\data\metadata.sqlite
Import fertig: C:\Projekte\fedcatalog_agent\catalog_agent\data\metadata.sqlite


In [10]:
import sqlite3, pandas as pd
con = sqlite3.connect(str((Path("C:/Projekte/fedcatalog_agent/catalog_agent/data/mock_energy_metadata.json").resolve().parent / "metadata.sqlite")))
print(pd.read_sql_query("SELECT COUNT(*) AS n FROM energy_metadata", con))
print(pd.read_sql_query("SELECT dataset_id, title FROM energy_metadata ORDER BY dataset_id LIMIT 5", con))
con.close()

     n
0  100
    dataset_id                     title
0  energy_0001         Ladedaten BW 2023
1  energy_0002   Verbrauchsdaten SN 2023
2  energy_0003   Speicherstatus NRW 2023
3  energy_0004         Ladedaten SH 2023
4  energy_0005  Verbrauchsdaten NRW 2022
